# Week 2 - Preprocessing, part 2

# 1. Lesson: None

# 2. Weekly graph question

The Storytelling With Data book mentions planning on a "Who, What, and How" for your data story.  Write down a possible Who, What, and How for your data, using the ideas in the book.

In [ ]:
"""
Topic: Building Recommendation Systems for eCommerce
Project link: https://drive.google.com/file/d/1O5wSSYrw84Dpv2iaR5XzMz1ll0LgXBYK/view

Who: The audience is the data science and product teams at a mid/large e-commerce or gaming platform 
companies like Amazon, Target, Olist, or Nintendo, Steam, or XBox. Specifically, I will target the 
lead data scientist who oversees recommendation algorithms, and a product manager responsible for personalization. 
These people are directly responsible for improving user engagement and conversion through smarter recommendations, 
so they would benefit most from exploratory insights that could uncover overlooked patterns or opportunities 
for better user data modelling.

What: The aim is to share findings from an exploratory analysis of three datasets — Steam user behavior, 
global game sales, and Olist's eCommerce logs - with a focus on patterns that could inform future improvements to 
recommendation systems. This might include things like content features that have been underutilized, regional 
preferences in genre, or trends in purchase timing. The goal is to bring new ideas to define where improvements in
recommendation systems could add value.

How: I can deliver a structured report (slideument) or Jupyter notebook with clear visuals, annotated takeaways, and a 
short executive summary. I'll compare how user interaction data, sales performance, and transactional behavior 
vary across platforms and contexts. The presentation will be geared toward sparking discussion about next steps 
in model development, not pitching a solution yet—just giving them fresh angles to think about.
"""

# 3. Homework - work with your own data

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import kagglehub
import os

This week, you will do the same types of exercises as last week, but you should use your chosen datasets that someone in your class found last semester. (They likely will not be the particular datasets that you found yourself.)

### Here are some types of analysis you can do  Use Google, documentation, and ChatGPT to help you:

- Summarize the datasets using info() and describe()

- Are there any duplicate rows?

- Are there any duplicate values in a given column (when this would be inappropriate?)

- What are the mean, median, and mode of each column?

- Are there any missing or null values?

    - Do you want to fill in the missing value with a mean value?  A value of your choice?  Remove that row?

- Identify any other inconsistent data (e.g. someone seems to be taking an action before they are born.)

- Encode any categorical variables (e.g. with one-hot encoding.)

### Conclusions:

- Are the data usable?  If not, find some new data!

- Do you need to modify or correct the data in some way?

- Is there any class imbalance?  (Categories that have many more items than other categories).

In [ ]:
olist_dataset_link = "https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce"
vgrecs_dataset_link = "https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam"
vgsales_dataset_link = "https://www.kaggle.com/datasets/ashaheedq/video-games-sales-2019"

olist_path = kagglehub.dataset_download(olist_dataset_link)
vgrec_path = kagglehub.dataset_download(vgrecs_dataset_link)
vgsales_path = kagglehub.dataset_download(vgsales_dataset_link)

# Video Game Sales DFs
vgsales_path_to_file = os.path.join(vgsales_path, "vgsales-12-4-2019-short.csv")
vgsales_df_raw = pd.read_csv(vgsales_path_to_file)

# Video Game Recommendation DFs
vgrecs_games_df_raw = pd.read_csv(os.path.join(vgrec_path, "games.csv"))
vgrecs_games_df_raw = pd.read_csv(os.path.join(vgrec_path, "games.csv"))
vgrecs_games_df_raw = pd.read_csv(os.path.join(vgrec_path, "games.csv"))

# Olist DFs
olist_df_raw = pd.read_csv(os.path.join(olist_path, "vgsales-12-4-2019-short.csv"))

# 4. Storytelling With Data graph

Just like last week: choose any graph in the Introduction of Storytelling With Data. Use matplotlib to reproduce it in a rough way. I don't expect you to spend an enormous amount of time on this; I understand that you likely will not have time to re-create every feature of the graph. However, if you're excited about learning to use matplotlib, this is a good way to do that. You don't have to duplicate the exact values on the graph; just the same rough shape will be enough.  If you don't feel comfortable using matplotlib yet, do the best you can and write down what you tried or what Google searches you did to find the answers.